In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import pi
import seaborn as sns
sns.set_theme(style = 'whitegrid')
from PIL import Image
import os

In [ ]:
class Solver:

    def __init__(self, h):
        self.gamme = 5. / 3
        self.L = 10
        self.T = 0.02
        self.maxCFL = 0.01

        self.v_l = 0
        self.rho_l = 13 # кг / м^3
        self.P_l = 10 * 101325 # Па
        self.v_r = 0
        self.rho_r = 1.3 
        self.P_r = 101325

        self.h = h

        self.tau = self.maxCFL * self.h
        self.t = self.tau
        
        self.NX = int(2 * self.L / self.h + 1)
        self.x_i = np.linspace(-self.L, self.L, self.NX)

        ## Начальные условия
        self.w_i = np.zeros((self.NX, 3))

        for i in range(self.NX // 2):
            self.w_i[i, 0] = self.rho_l
            self.w_i[i, 2] = self.P_l / (self.gamma - 1)

        for i in range((self.NX // 2), self.NX):
            self.w_i[i, 0] = self.rho_r
            self.w_i[i, 2] = self.P_r / (self.gamma - 1)

        self.v_i = np.zeros((3, self.NX))
        self.P_i = np.zeros(self.NX)
        self.v_i[0, :] = self.w_i[0, :] 
        self.v_i[1, :] = self.w_i[1, :]
        self.v_i[2, :] = self.w_i[2, :] / self.w_i[0, :]
        self.P_i = (self.gamma - 1) * self.w_i[2, :] 

        ## Графики

        ## Плотность
        plt.figure(figsize=(10, 6))
        plt.scatter(self.x_i, self.v_i[0, :], color = 'grey')
        plt.ylim([self.v_i[0, :].min(), self.v_i[0, :].max()])
        plt.title(f'rho(t)\nt = 0, h = {self.h}',
                fontsize=14)
        if not os.path.exists(f'./img/rho'):
                    os.mkdir(f'./img/rho')
        plt.savefig(f'./img/rho/img_0.png', 
                transparent = False,  
                facecolor = 'white'
                )
        plt.close()
        ## Скорость
        plt.figure(figsize=(10, 6))
        plt.scatter(self.x_i, self.v_i[1, :], color = 'grey')
        plt.ylim([self.v_i[1, :].min(), self.v_i[1, :].max()])
        plt.title(f'u(t)\nt = 0, h = {self.h}',
                fontsize=14)
        if not os.path.exists(f'./img/u'):
                    os.mkdir(f'./img/u')
        plt.savefig(f'./img/u/img_0.png', 
                transparent = False,  
                facecolor = 'white'
                )
        plt.close()
        ## Энергия
        plt.figure(figsize=(10, 6))
        plt.scatter(self.x_i, self.v_i[2, :], color = 'grey')
        plt.ylim([self.v_i[2, :].min(), self.v_i[2, :].max()])
        plt.title(f'e(t)\nt = 0, h = {self.h}',
                fontsize=14)
        if not os.path.exists(f'./img/e'):
                    os.mkdir(f'./img/e')
        plt.savefig(f'./img/e/img_0.png', 
                transparent = False,  
                facecolor = 'white'
                )
        plt.close()
        ## Давление
        plt.figure(figsize=(10, 6))
        plt.scatter(self.x_i, self.P_i, color = 'grey')
        plt.ylim([self.P_i.max(), self.P_i.min()])
        plt.title(f'P(t)\nt = 0, h = {self.h}',
                fontsize=14)
        if not os.path.exists(f'./img/P'):
                    os.mkdir(f'./img/P')
        plt.savefig(f'./img/P/img_0.png', 
                transparent = False,  
                facecolor = 'white'
                )
        plt.close()

        ## Массив следующего слоя по времени
        self.w_i_next = np.zeros((self.NX, 3))

    def solveAndCreateGif(self):
        while self.t <= self.T:
            for i in range(1, self.NX - 1):
                c = np.sqrt(self.gamma * (self.gamma - 1) * self.v_i[2, i])
                omega_T = np.zeros((3,3))
                u = self.v[1, i]
                omega_T[0, 0] = -u * c 
                omega_T[0, 1] = c 
                omega_T[0, 2] = u - 1
                omega_T[1, 0] = -c * c
                omega_T[1, 2] = self.gamma - 1
                omega_T[2, 0] = u * c
                omega_T[2, 1] = -c
                omega_T[2, 2] = self.gamma - 1

                omega_T_inv = np.linalg.inv(omega_T)

                lmbda = np.zeros((3, 3))
                lmbda[0, 0] = u + c
                lmbda[1, 1] = u
                lmbda[2, 2] = u - c

                lmbda_abs = np.abs(lmbda)
                A = np.zeros((3, 3))
                A[0, 1] = 1
                A[1, 0] = -u * u
                A[1, 1] = 2 * u
                A[1, 2] = self.gamma - 1
                A[2, 0] = - self.gamma * u * self.v_i[2, i]


                self.w_i_next[:, i] =  self.w_i[:, i] - self.tau 
                
            
            plt.figure(figsize=(10, 6))
            plt.scatter(self.x_i, self.u_next_m, color = 'grey')
            plt.ylim([-1.2, 1.2])
            plt.title(f'{self.method_name}\nt = {self.t_i[t]:.2f}, CFL = {self.CFL}, h = {self.h}',
            fontsize=14)
            plt.savefig(f'./img/{self.method_name}_CFL{self.CFL}_h{self.h}/img_{self.t}.png', 
            transparent = False,  
            facecolor = 'white'
            )
            plt.close()

            self.u_n_m = self.u_next_m.copy()

        image_path_list = [f'./img/{self.method_name}_CFL{self.CFL}_h{self.h}/img_{i}.png' for i in range(self.TX)]
        image_list = [Image.open(file) for file in image_path_list]
        image_list[0].save(
                f'gifs/{self.method_name}_CFL{self.CFL}_h{self.h}.gif',
                save_all=True,
                append_images=image_list[1:], # append rest of the images
                duration=0.08, # in milliseconds
                loop=0)
        
    def setCFL(self, cfl):
        self.CFL = cfl

    def setH(self, h):
        self.h = h

    def setL(self, L):
        self.L = L

    def setT(self, T):
        self.T = T

    def reInit(self):
        self.tau = self.CFL * self.h

        self.NX = int(self.L / self.h + 1)
        self.TX = int(self.T / self.tau + 1) 

        self.x_i = np.array([(i - 1) * self.h for i in range(1, self.NX + 1)])

        self.u_n_m = self.u(self.x_i, self.L)
        self.u_next_m = self.u_n_m.copy()